# Uporaba PostgreSQL v Pythonu s knjižnico `psycopg2`

## Nameščanje knjižnice `psycopg2`

Namestitev izvedemo iz ukazne vrstice. Če nimamo administrativnih pravic na racunalniku, izvedemo:

```bash
python -m pip install --user psycopg2
```


Če jih imamo:
```bash
python -m pip install psycopg2
```

**Pozor:** za uporabo tega programa moramo imeti dostop do podatkovne baze PostgreSQLs pravico ustvarjanja tabel. Bazo PostgreSQL si lahko namestimo na svojem racunalniku in v datoteki `auth.py` nastavimo ustrezne podatke za dostop.

In [ ]:
import psycopg2
from auth import auth

Ustvarimo tabelo in jo napolnimo.

In [ ]:
with psycopg2.connect(**auth) as con:
    cur = con.cursor()   # "odzivnik" za pregledovanje poizvedbe
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name TEXT, Price INTEGER)")
    cur.execute("INSERT INTO Cars VALUES (1, 'Audi', 52642)")
    cur.execute("INSERT INTO Cars VALUES (2, 'Mercedes', 57127)")
    cur.execute("INSERT INTO Cars VALUES (3, 'Skoda', 9000)")
    cur.execute("INSERT INTO Cars VALUES (4, 'Volvo', 29000)")
    cur.execute("INSERT INTO Cars VALUES (5, 'Bentley', 350000)")
    cur.execute("INSERT INTO Cars VALUES (6, 'Citroen', 21000)")
    cur.execute("INSERT INTO Cars VALUES (7, 'Hummer', 41400)")
    cur.execute("INSERT INTO Cars VALUES (8, 'Volkswagen', 21600)")

Povezavo si lahko tudi shranimo in večkrat uporabimo.

In [ ]:
con = psycopg2.connect(**auth)

Primer: Katere vrstice so v tabeli?

In [ ]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    print(cur.fetchall())

Ko kurzor enkrat vrne podatke, se "izčrpa".

In [ ]:
cur.fetchall()

Z `with` smo naredili transakcijo - ob uspešnem koncu se je potrdila, sicer bi se preklicala.

In [ ]:
with con:
    cur.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")
    cur.execute("INSERT INTO Cars VALUES (9, 'Ferrari', 192000)")

Transakcije lahko potrjujemo in preklicujemo tudi sami.

In [ ]:
cur.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")

In [ ]:
with psycopg2.connect(**auth) as con2: # uporabimo novo povezavo
    cur2 = con2.cursor()
    cur2.execute("SELECT * from Cars")
    print(cur2.fetchall())

In [ ]:
con.rollback()

In [ ]:
con.commit()

Primer: Poizvedba z iteracijo po 'odzivniku'.

In [ ]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    for podatek in cur:
        print(podatek)

Primer interaktivne poizvedbe,  uporaba niza `%s`.

In [ ]:
with con:
    cur = con.cursor()
    kaj = input("Cena katerega vozila te zanima:")
    cur.execute("SELECT Name, Price FROM Cars WHERE Name LIKE %s", (kaj,))
    print(cur.fetchall())

Primer interaktivne poizvedbe, uporaba `%(ključ)s`, preverjanje rezultata.

In [ ]:
with con:
    cur = con.cursor()
    kaj = input("Katero vozilo te zanima:").strip()
    cur.execute("SELECT Price FROM Cars WHERE Name LIKE %(ime)s", {"ime": kaj})
    rez = cur.fetchone()
    if rez is None:
        print("Ni takega vozila ({0}).".format(kaj))
    else:
        print("Cena vozila {0} je {1}.".format(kaj, rez[0]))

Primer: večkratno vstavljanje.

In [ ]:
cars = [
    (9, 'Audi2', 52643),
    (10, 'MercedesX', 57642),
    (11, 'Škoda', 9000),
    (12, 'Volvek', 29000),
    (13, 'Bentley7', 350000),
    (14, 'HummerU', 41400),
    (15, 'VolkswagenPassat', 21600)
]
print("Vstavljamo: ", cars)
with con:
    cur = con.cursor()
    cur.executemany("INSERT INTO Cars VALUES (%s, %s, %s)", cars)
    cur.execute("SELECT * from Cars")
    print("V tabeli imamo:")
    print(cur.fetchall())

Ko povezave več ne potrebujemo, jo zapremo.

In [ ]:
con.close()